In [3]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.axes as ax
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.base.model import LikelihoodModel
from sklearn import preprocessing

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
import unicodedata
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [47]:
def subID_list():
    subID = ['797','806','809','826','843','845','855','856','857','1031','1153','1154','1157','1174','1262','1264','1266',
        '1364','1367','1379','1380','1381','1384','1435','1465','1467','1468','1489','1492','1493','1494','1495','1498','1519',
        '1520','1527','1532','1536','1541','1721','1722','1737','1741','1788','1791','1810','1812','1833','1934',
            '2310','2316','2318','2323','2338','2339','2340','2348','2353','2356','2357','2358','2377','2378',
             '2381','2383','2385','2386','2387','2388','2389','2390','2391','2394','2395','2396','2399',
             '2401','2402','2404','2405','2407','2434','2444','2456',
            '2433','2447','2477','2479','2489','2547','2548','2549','2553','2555','2558','2559','2578','2951','2958',
            '2961','2962','2963','2964','2967','2968','2975','2977','2978','2981','2983',
             '2998','2999','3051','3052','3053']
    return subID
subID = subID_list()
print(len(subID))

115


In [48]:
iv_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Independent_variable_1.csv")
dv_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Dependent_variable.csv")
ratio_rt_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/ratio_rt_df.csv")
age_gender_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/age_gender_df.csv")

df = iv_df.drop(['Unnamed: 0','E_RP','F_RP','S_RP','R_RP','S_RP.1','E_EB','F_EB','S_EB','R_EB','S_EB.1'], axis = 1)

DV = dv_df[['subID','alpha_EU','gamma_EU','alpha_ESVT','M_ESVT','gamma_ESVT']]
DV.loc[:,'RT'] = ratio_rt_df['RT']
DV.loc[:,'risk_Ratio'] = ratio_rt_df['risk_ratio']

df.loc[:,'alpha_EU']= DV['alpha_EU']
df.loc[:,'gamma_EU'] = DV['gamma_EU']
df.loc[:,'alpha_ESVT'] = DV['alpha_ESVT']
df.loc[:,'M_ESVT'] = DV['M_ESVT']
df.loc[:,'gamma_ESVT']  = DV['gamma_ESVT']
df.loc[:,'RT'] = DV['RT']
df.loc[:,'risk_Ratio'] = DV['risk_Ratio']
df.loc[:,'gender'] = age_gender_df['Gender']
df.loc[:,'age'] = age_gender_df['Age']

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### R squared < 0.1 
#### 843,  856, 857, 1031, 1465, 1520. 2340, 2378, 2387, 2401, 2444, 2447
### No risk option (below 0.2)
#### 1519, 1381, 1435,1493, 1721
### Always risk
#### 1492, 3053

In [49]:
subID = subID_list()
R = ['843','856','857','1031','1381','1435','1465','1492',
     '1493','1519','1520','1721','2340','2378','2387','2401','2444','2447','3053']
N = [i for i in range(len(subID)) if subID[i] in R]

df = df[~df.subID.isin(R)]
censored_df = df.drop(['subID'], axis = 1)

0       797
1       806
2       809
3       826
5       845
6       855
10     1153
11     1154
12     1157
13     1174
14     1262
15     1264
16     1266
17     1364
18     1367
19     1379
20     1380
22     1384
25     1467
26     1468
27     1489
30     1494
31     1495
32     1498
35     1527
36     1532
37     1536
38     1541
40     1722
41     1737
       ... 
83     2456
84     2433
86     2477
87     2479
88     2489
89     2547
90     2548
91     2549
92     2553
93     2555
94     2558
95     2559
96     2578
97     2951
98     2958
99     2961
100    2962
101    2963
102    2964
103    2967
104    2968
105    2975
106    2977
107    2978
108    2981
109    2983
110    2998
111    2999
112    3051
113    3052
Name: subID, Length: 96, dtype: int64


In [63]:
censored_df['subID'] = df['subID']
censored_df.iloc[:,0:30]

,E_LI,F_LI,S_LI,R_LI,S_LI.1,IUS,lot_pos,lot_neg,drm_pos,drm_neg,...,crt,alpha_EU,gamma_EU,alpha_ESVT,M_ESVT,gamma_ESVT,RT,risk_Ratio,gender,age
0,14.0,15.0,14.0,10.0,24.0,62.0,9.0,12.0,3.063750,1.823125,...,1.0,0.642496,3.288654,1.000000,25.446097,40.442038,106.3125,0.41,0,26
1,10.0,13.0,15.0,15.0,13.0,69.0,7.0,9.0,3.916250,1.135000,...,3.0,0.690595,1.831889,1.000000,43.222091,70.515801,124.3750,0.53,0,27
2,23.0,16.0,23.0,17.0,36.0,96.0,9.0,11.0,4.333125,1.406250,...,3.0,0.631411,1.828818,1.000000,26.177699,16.318587,101.3750,0.41,0,25
3,18.0,20.0,22.0,26.0,33.0,73.0,10.0,11.0,2.687500,2.093125,...,2.0,0.591172,4.175422,1.000000,19.008396,30.809443,89.7500,0.35,1,27
5,18.0,17.0,23.0,12.0,30.0,81.0,12.0,7.0,5.250000,0.426875,...,1.0,0.721169,2.026993,1.000000,41.031170,33.051698,195.8125,0.47,0,38
6,21.0,13.0,32.0,19.0,38.0,108.0,10.0,11.0,3.291250,0.729375,...,1.0,0.642062,1.618877,1.972455,9.703613,45.996255,245.3750,0.65,1,51
10,15.0,23.0,16.0,21.0,22.0,51.0,10.0,11.0,3.500000,1.365000,...,2.0,0.560818,3.025380,1.000000,19.008396,30.809443,188.2500,0.35,1,31
11,15.0,21.0,19.0,13.0,23.0,102.0,8.0,11.0,3.250000,2.458750,...,NaN,1.072192,0.202508,2.542819,341.055370,2490.345009,174.0000,0.65,1,65
12,21.0,20.0,27.0,14.0,30.0,105.0,10.0,12.0,3.208125,1.501875,...,0.0,1.551923,0.051672,7.150716,9.105696,33.403640,146.0625,1.00,0,32
13,19.0,13.0,19.0,17.0,31.0,65.0,9.0,8.0,3.436875,2.603750,...,0.0,0.473414,1.488877,1.000000,9.445408,10.018487,141.5625,0.29,0,39


In [45]:
x= censored_df.dropna().to_numpy()
# Standardizing the features
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2','PC3','PC4'])
pca.explained_variance_ratio_
feature_imp = pca.components_
loading_matrix_linear_com = pd.DataFrame(feature_imp.T, columns=['PC1', 'PC2', 'PC3','PC4'])
loading_matrix_linear_com

,PC1,PC2,PC3,PC4
0,0.076289,0.240295,0.064219,-0.115861
1,-0.057648,0.279102,-0.084877,0.130428
2,0.008479,0.346319,0.120831,-0.087601
3,-0.000130,0.307205,0.154329,-0.119563
4,-0.081630,0.277701,0.182743,-0.251822
5,0.286390,0.004313,-0.072677,0.051268
6,-0.242621,0.211073,-0.204924,-0.065897
7,0.277287,-0.044653,0.092200,-0.035013
8,-0.234216,0.164621,-0.286650,0.042061
9,0.295172,0.133556,-0.028900,0.057402


In [38]:
risk_ratio_weekly = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ratio_weekly.csv")

risk_EU_alpha_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_EU_alpha_weekly.csv")
risk_EU_gamma_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_EU_gamma_weekly.csv")
risk_ESVT_alpha_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_alpha_weekly.csv")
risk_ESVT_M_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_M_weekly.csv")
risk_ESVT_gamma_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_gamma_weekly.csv")

risk_ratio_weekly = risk_ratio_weekly[~risk_ratio_weekly['Unnamed: 0'].isin(R)]
risk_EU_alpha_weekly  = risk_EU_alpha_weekly[~risk_EU_alpha_weekly['Unnamed: 0'].isin(R)]
risk_EU_gamma_weekly  = risk_EU_gamma_weekly[~risk_EU_gamma_weekly['Unnamed: 0'].isin(R)]
risk_ESVT_alpha_weekly  = risk_ESVT_alpha_weekly[~risk_ESVT_alpha_weekly['Unnamed: 0'].isin(R)]
risk_ESVT_M_weekly  = risk_ESVT_M_weekly[~risk_ESVT_M_weekly['Unnamed: 0'].isin(R)]
risk_ESVT_gamma_weekly  = risk_ESVT_gamma_weekly[~risk_ESVT_gamma_weekly['Unnamed: 0'].isin(R)]






In [64]:

week = [1,2,3,4,5,6,7,8]
pc3_score = np.zeros((len(week),96))
for i in range(len(week)):
    censored_df['risk_Ratio'] = risk_ratio_weekly.iloc[:,i+1]
    censored_df['alpha_EU'] = risk_EU_alpha_weekly.iloc[:,i+1]
    censored_df['gamma_EU'] = risk_EU_gamma_weekly.iloc[:,i+1]
    censored_df['alpha_ESVT'] = risk_ESVT_alpha_weekly.iloc[:,i+1]
    censored_df['M_ESVT'] = risk_ESVT_M_weekly.iloc[:,i+1]
    censored_df['gamma_ESVT'] = risk_ESVT_gamma_weekly.iloc[:,i+1]
    
    #print(i)
    pc3 = loading_matrix_linear_com['PC3']
    e = np.array(censored_df.iloc[:,0:30]).dot(np.array(pc3))
    pc3_score[i,:] = e

In [70]:
pc3_score
pc3_score_df = pd.DataFrame(data=pc3_score.T,columns = ['week1','week2','week3','week4','week5','week6','week7','week8'])
pc3_score_df.insert(0, 'subID', censored_df['subID'].tolist())



In [76]:
pc3_score_df.to_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/pc3_score_cencored.csv")